In [1]:
import pandas as pd
from conexion import connect_to_postgresql

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
provincias = {
    "Ávila": [30869, 30870, 30877, 37683],
    "Burgos": [30926, 30927, 30934, 37687],
    "León": [31070, 31071, 31078, 37703],
    "Palencia": [31142, 31143, 31150, 37708],
    "Salamanca": [31178, 31179, 31186, 37713],
    "Segovia": [31196, 31197, 31204, 37715],
    "Soria": [31214, 31215, 31222, 37717],
    "Valladolid": [31259, 31260, 31267, 37722],
    "Zamora": [31268, 31269, 31276, 37723],
}

In [4]:
def procesar_datos_cyl(provincias: dict) -> pd.DataFrame:
    indicadores_cyl = None  # Aquí uniremos los datos de todas las provincias

    for provincia, codigos in provincias.items():
        indicadores_provincia = None  # Para cada provincia

        for codigo in codigos:
            try:
                df = pd.read_csv(f'raw_data/municipios/{provincia}/{codigo}.csv', encoding='utf-8', sep=';')
            except FileNotFoundError:
                print(f"Error: Archivo {codigo}.csv no encontrado en {provincia}.")
                continue
            except pd.errors.ParserError as e:
                print(f"Error al parsear {codigo}.csv en {provincia}: {e}")
                continue
            except Exception as e:
                print(f"Un error inesperado ocurrió con {codigo}.csv en {provincia}: {e}")
                continue

            df['Provincia'] = provincia
            df = df.rename(columns={'Municipios': 'Municipio'})
            df.fillna(0, inplace=True)
            df = df[(df['Distritos'] == 0) & (df['Secciones'] == 0)]
            df.drop(columns=['Distritos', 'Secciones'], inplace=True)
            
            if codigo == codigos[0]:  # Indicadores de renta
                df['Total'] = df['Total'].str.replace('.', '').str.replace(',', '.')
                df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
                df = df.pivot_table(index=['Periodo', 'Municipio', 'Provincia'],
                                    columns='Indicadores de renta media y mediana',
                                    values='Total')
            elif codigo == codigos[1]:  # Fuente de ingresos
                df['Total'] = df['Total'].str.replace('.', '').str.replace(',', '.')
                df['Total'] = pd.to_numeric(df['Total'])
                df = df.pivot_table(index=['Periodo', 'Municipio', 'Provincia'],
                                    columns='Distribución por fuente de ingresos',
                                    values='Total')
            elif codigo == codigos[2]:  # Indicadores demográficos
                df['Total'] = df['Total'].str.replace('.', '').str.replace(',', '.')
                df['Total'] = pd.to_numeric(df['Total'])
                df = df.pivot_table(index=['Periodo', 'Municipio', 'Provincia'],
                                    columns='Indicadores demográficos',
                                    values='Total')
            elif codigo == codigos[3]:  # Índice de Gini
                df['Total'] = df['Total'].str.replace('.', '').str.replace(',', '.')
                df['Total'] = pd.to_numeric(df['Total'])
                df = df.pivot_table(index=['Periodo', 'Municipio', 'Provincia'],
                                    columns='Índice de Gini y Distribución de la renta P80/P20',
                                    values='Total')

            df.reset_index(inplace=True)
            df[['CP', 'Municipio']] = df['Municipio'].str.split(' ', n=1, expand=True)

            if indicadores_provincia is None:
                indicadores_provincia = df
            else:
                indicadores_provincia = indicadores_provincia.join(df.set_index(['Periodo', 'CP', 'Municipio', 'Provincia']),
                                                                   on=['Periodo', 'CP', 'Municipio', 'Provincia'],
                                                                   how='left',
                                                                   lsuffix='_prev',
                                                                   rsuffix=f'_{codigo}')

        if indicadores_cyl is None:
            indicadores_cyl = indicadores_provincia
        else:
            indicadores_cyl = pd.concat([indicadores_cyl, indicadores_provincia])

    return indicadores_cyl


In [5]:
indicadores_cyl = procesar_datos_cyl(provincias)
indicadores_cyl.head()

,Periodo,Municipio,Provincia,Media de la renta por unidad de consumo,Mediana de la renta por unidad de consumo,Renta bruta media por hogar,Renta bruta media por persona_prev,Renta neta media por hogar,Renta neta media por persona,CP,Fuente de ingreso: otras prestaciones,Fuente de ingreso: otros ingresos,Fuente de ingreso: pensiones,Fuente de ingreso: prestaciones por desempleo,Fuente de ingreso: salario,Renta bruta media por persona_30870,Edad media de la población,Población,Porcentaje de hogares unipersonales,Porcentaje de población de 65 y más años,Porcentaje de población española,Porcentaje de población menor de 18 años,Tamaño medio del hogar,Distribución de la renta P80/P20,Índice de Gini,Renta bruta media por persona_30927,Renta bruta media por persona_31071,Renta bruta media por persona_31143,Renta bruta media por persona_31179,Renta bruta media por persona_31197,Renta bruta media por persona_31215,Renta bruta media por persona_31260,Renta bruta media por persona_31269
0,2015,Adanero,Ávila,14207.0,13650.0,25042.0,11764.0,22162.0,10411.0,05001,545.0,2431.0,3103.0,399.0,5286.0,11764.0,51.0,227.0,44.5,31.7,95.9,9.3,2.1,2.4,28.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,"Adrada, La",Ávila,11311.0,9450.0,20974.0,8758.0,18513.0,7730.0,05002,347.0,969.0,2321.0,407.0,4714.0,8758.0,42.4,2593.0,38.0,18.8,90.7,18.9,2.4,3.6,36.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,Albornos,Ávila,13028.0,12250.0,21550.0,10128.0,19550.0,9189.0,05005,188.0,1800.0,2912.0,290.0,4938.0,10128.0,50.7,206.0,45.0,30.6,99.4,10.2,2.1,2.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,Aldeanueva de Santa Cruz,Ávila,10477.0,8750.0,17142.0,7665.0,16620.0,7432.0,05007,771.0,1995.0,3353.0,81.0,1481.0,7665.0,56.1,128.0,38.2,41.2,98.3,8.1,2.2,2.2,25.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,Aldeaseca,Ávila,12457.0,11200.0,24442.0,8990.0,22191.0,8162.0,05008,482.0,901.0,2990.0,225.0,4392.0,8990.0,52.2,237.0,30.9,34.6,96.6,8.0,2.5,3.1,31.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def unificar_columnas(df, patron, nueva_columna):
    """
    Unifica varias columnas en una sola en función de un patrón dado.

    Args:
        df (pd.DataFrame): DataFrame original.
        patron (str): Patrón para identificar las columnas a unificar (e.g., "Renta bruta media por persona").
        nueva_columna (str): Nombre de la nueva columna unificada.

    Returns:
        pd.DataFrame: DataFrame con la columna unificada.
    """
    # Identificar columnas que coinciden con el patrón
    columnas_a_unificar = [col for col in df.columns if col.startswith(patron)]
    
    df[nueva_columna] = df[columnas_a_unificar].sum(axis=1, skipna=True)
    
    df = df.drop(columns=columnas_a_unificar)
    
    return df

In [7]:
indicadores_cyl = unificar_columnas(indicadores_cyl, "Renta bruta media por persona_", "Renta bruta media por persona")
indicadores_cyl.head()

,Periodo,Municipio,Provincia,Media de la renta por unidad de consumo,Mediana de la renta por unidad de consumo,Renta bruta media por hogar,Renta neta media por hogar,Renta neta media por persona,CP,Fuente de ingreso: otras prestaciones,Fuente de ingreso: otros ingresos,Fuente de ingreso: pensiones,Fuente de ingreso: prestaciones por desempleo,Fuente de ingreso: salario,Edad media de la población,Población,Porcentaje de hogares unipersonales,Porcentaje de población de 65 y más años,Porcentaje de población española,Porcentaje de población menor de 18 años,Tamaño medio del hogar,Distribución de la renta P80/P20,Índice de Gini,Renta bruta media por persona
0,2015,Adanero,Ávila,14207.0,13650.0,25042.0,22162.0,10411.0,05001,545.0,2431.0,3103.0,399.0,5286.0,51.0,227.0,44.5,31.7,95.9,9.3,2.1,2.4,28.5,23528.0
1,2015,"Adrada, La",Ávila,11311.0,9450.0,20974.0,18513.0,7730.0,05002,347.0,969.0,2321.0,407.0,4714.0,42.4,2593.0,38.0,18.8,90.7,18.9,2.4,3.6,36.3,17516.0
2,2015,Albornos,Ávila,13028.0,12250.0,21550.0,19550.0,9189.0,05005,188.0,1800.0,2912.0,290.0,4938.0,50.7,206.0,45.0,30.6,99.4,10.2,2.1,2.0,27.0,20256.0
3,2015,Aldeanueva de Santa Cruz,Ávila,10477.0,8750.0,17142.0,16620.0,7432.0,05007,771.0,1995.0,3353.0,81.0,1481.0,56.1,128.0,38.2,41.2,98.3,8.1,2.2,2.2,25.5,15330.0
4,2015,Aldeaseca,Ávila,12457.0,11200.0,24442.0,22191.0,8162.0,05008,482.0,901.0,2990.0,225.0,4392.0,52.2,237.0,30.9,34.6,96.6,8.0,2.5,3.1,31.4,17980.0


In [8]:
try:
    engine = connect_to_postgresql()
    conn = engine.raw_connection()
    cur = conn.cursor()
    print("Empieza la carga indicadores_cyl_raw")
    indicadores_cyl.to_sql('indicadores_cyl_raw', engine, if_exists='replace', index=False)
    print("Carga de indicadores_cyl_raw correcta")
except Exception as e:
    print(f"Las tablas no se han podido cargar debido a: {str(e)}")
finally:
    if conn:
        conn.close()
        print("Conexión cerrada correctamente.")



Conexión establecida
Empieza la carga indicadores_cyl_raw
Carga de indicadores_cyl_raw correcta
Conexión cerrada correctamente.


In [9]:
# import psycopg2
# 
# # Establecer conexión usando psycopg2
# try:
#     conn = psycopg2.connect(
#         dbname="tfm",
#         user="postgres",
#         password="CHUKy-18",
#         host="localhost",
#         port="5432"
#     )
#     print("Conexión establecida")
# except Exception as e:
#     print(f"Failed to connect: {e}")
# 
# # Llamar a la función para cada dataframe y tabla
# try:
#     load_data_with_tempfile(indicadores_cyl, 'indicadores_CyL_raw', conn)
# finally:
#     if conn:
#         conn.close()
#         print("Conexión cerrada")
# 